In [ ]:
# 导入相应的模块
import numpy as np
from statsmodels import regression
import statsmodels.api as sm
import matplotlib.pyplot as plt
import math

In [ ]:
start_date = '2019-06-12'
end_date = '2021-07-01'
asset4_df = DataSource('bar1d_CN_FUND').read(instruments=['512760.HOF']).set_index('date',drop=False)#半导体场内基金
asset4 = asset4_df[(asset4_df['date']>=start_date)&(asset4_df['date']<=end_date)]
# 波动率计算
asset4['qianfuquan'] = asset4['close']*asset4['adjust_factor']
asset4['shift'] = asset4['qianfuquan'].shift(axis=0) #前一天的close
asset4['difference'] = np.log(asset4['qianfuquan'])-np.log(asset4['shift'])
asset4['mean'] = asset4['difference'].rolling(10).mean()
asset4['volatility'] = asset4['difference'].rolling(10).std()*np.sqrt(252)
asset4['quantile80'] = asset4['volatility'].rolling(120).quantile(0.8)
asset4['quantile50'] = asset4['volatility'].rolling(120).quantile(0.5)

asset4['dif_quantile'] = asset4['volatility'] / asset4['quantile80']
asset4['dif_quantile_2'] = asset4['volatility'] - asset4['quantile50']
asset4['beta_accept_line'] = asset4['dif_quantile_2'].apply(lambda x : 0 if x<0 else 1) #位于历史波动率50%水平以下时不对冲beta，位于50%氛围以上时配置
asset4['beta_kill_preference'] = asset4['dif_quantile'].apply(lambda x: 0.323 if x>=1 else 0) #在波动超过历史80%阈值时加大对冲力度,0.323可自定义设置
asset4['beta_kill'] = asset4['beta_accept_line']+asset4['beta_kill_preference']
# asset4['volatility_change'] = asset4['volatility'].pct_change()[1:]

print(sum(asset4['beta_kill']))
asset4['volatility'].plot(figsize=[9,6])
asset4['qianfuquan'].pct_change()[1:].plot()
asset4['qianfuquan'].plot()

In [ ]:
# 获取一段时间的股票数据
# start_date = '2020-02-10'
# end_date = '2021-07-01'
# 策略选择三个收益率数据
# asset1 = D.history_data('600519.SHA',start_date,end_date,fields=['close']).set_index('date')['close']
# asset2 = D.history_data('300285.SZA',start_date,end_date,fields=['close']).set_index('date')['close']
# asset3 = D.history_data('515030.HOF',start_date,end_date,fields=['close']).set_index('date')['close']
# asset4 = D.history_data('512480.HOF',start_date,end_date,fields=['close']).set_index('date')['close']

# asset1_df = DataSource('bar1d_CN_FUND').read(instruments=['512690.HOF']).set_index('date',drop=False)#白酒
# asset1 = asset1_df[(asset1_df['date']>=start_date)&(asset1_df['date']<=end_date)]['close']
# asset2_df = DataSource('bar1d_CN_FUND').read(instruments=['512170.HOF']).set_index('date',drop=False)#医药
# asset2 = asset2_df[(asset2_df['date']>=start_date)&(asset2_df['date']<=end_date)]['close']
# asset3_df = DataSource('bar1d_CN_FUND').read(instruments=['515700.HOF']).set_index('date',drop=False)#新能源车
# asset3 = asset3_df[(asset3_df['date']>=start_date)&(asset3_df['date']<=end_date)]['close']
# asset5 = D.history_data('300285.SZA',start_date,end_date,fields=['close']).set_index('date')['close'] #电子-被动元器件 代表
# asset6 = D.history_data('601012.SHA',start_date,end_date,fields=['close']).set_index('date')['close'] #光伏 代表
# asset7 = D.history_data('300274.SZA',start_date,end_date,fields=['close']).set_index('date')['close'] #光伏 代表
# asset8 = D.history_data('002241.SZA',start_date,end_date,fields=['close']).set_index('date')['close'] #电子-消费电子 代表

benchmark = D.history_data('000300.SHA',start_date,end_date,fields=['close']).set_index('date')['close']

# asset.name = 'portfolio'
benchmark.name = '000300.SHA'

# 计算收益率
# r_a = asset1.pct_change()[1:]*0.2+asset2.pct_change()[1:]*0.2+asset3.pct_change()[1:]*0.2+asset4.pct_change()[1:]*0.2+asset5.pct_change()[1:]*0.05+asset6.pct_change()[1:]*0.05+asset7.pct_change()[1:]*0.05+asset8.pct_change()[1:]*0.05
# r_a = asset7.pct_change()[1:]
r_a = asset4['qianfuquan'].pct_change()[1:] # 构建阿尔法策略
# print(r_a)
r_b = benchmark.pct_change()[1:]

# 绘制
r_a.plot(figsize=[9,6])
r_b.plot()
plt.ylabel("Daily Return")
plt.legend();


In [ ]:
X = r_b.values  
Y = r_a.values
x = sm.add_constant(X)
 
def linreg(x,y):
    # 增加一个常数项
    x = sm.add_constant(x)
    model = regression.linear_model.OLS(y,x).fit()
    # 再把常数项去掉
    x = x[:, 1]
    return model.params[0], model.params[1]

alpha, beta = linreg(X,Y)
print('alpha: ' + str(alpha))
print('beta: ' + str(beta))

In [ ]:
X2 = np.linspace(X.min(), X.max(), 100)
Y_hat = X2 * beta + alpha

plt.scatter(X, Y, alpha=0.3) # 画出原始数据散点
plt.xlabel("000300.SHA Daily Return")
plt.ylabel("600519.SHA Daily Return")

 # 增加一条红色的回归直线
plt.plot(X2, Y_hat, 'r', alpha=0.9);

In [ ]:
# 构建一个市场中性组合
beta_kill = asset4['beta_kill'][1:]

portfolio = -beta_kill*beta*r_b + r_a
portfolio.name = "portfolio + Hedge"

# 绘制各自的收益曲线
portfolio.plot(alpha=0.9,figsize=[9,6])
# r_b.plot(alpha=0.5);
r_a.plot(alpha=0.5);
plt.ylabel("Daily Return")
plt.legend();

In [ ]:
portfolio_adjust = list(portfolio+1)
multi = 1
port_multi_price = [1]
for i in portfolio_adjust:
    multi = multi*i
    port_multi_price.append(multi)
print('portfolio return: ', multi)

r_a_adjust = list(r_a+1)
multi = 1
r_a_multi_price = [1]
for i in r_a_adjust:
    multi = multi*i
    r_a_multi_price.append(multi)
print('origin return: ', multi)

asset4['portfolio_return'] = port_multi_price
asset4['origin_return'] = r_a_multi_price
asset4['origin_return'].plot(figsize=[9,6])
asset4['portfolio_return'].plot()
asset4['volatility'].plot()